In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

print(housing.feature_names)

from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, test_size = 0.2, random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.3, random_state=42)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

np.random.seed(42)
tf.random.set_seed(42)

#### Simple sequential model

In [ ]:
from tensorflow import keras
model = keras.models.Sequential()
model.add(keras.layers.Dense(30, input_dim=X_train.shape[1], activation='relu'))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(loss='mse',
             optimizer=keras.optimizers.SGD(lr=1e-3),
             metrics=['mae'])

## Initially let's run for 20 epochs
model_history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
mae_test = model.evaluate(X_test, y_test)

In [ ]:
model_history.history

In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

plt.show()
## So we can see that there's still scope to tune the model and achieve better loss values

In [ ]:
# Let's now run for 60 more epochs
model_history = model.fit(X_train, y_train, epochs=60, validation_data=(X_valid, y_valid))

In [ ]:
mae_test = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

plt.show()
## So we can see that there's still scope to tune the model and achieve better loss values

In [ ]:
## Let's try 60 more epochs
model_history = model.fit(X_train, y_train, epochs=60, validation_data=(X_valid, y_valid))

In [ ]:
import pydot
tf.keras.utils.plot_model(model)

In [ ]:
from ann_visualizer.visualize import ann_viz
ann_viz(model,title="model_neural_network")

In [ ]:
keras.utils.plot_model(model,rankdir='LR',show_shapes=True)

In [ ]:
mae_test = model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

plt.show()

In [ ]:
## Let's to predictions
X_new = X_test[:3]
y_pred = model.predict(X_new)
print(y_pred)
print(y_test[:3])

In [ ]:
## Notice that our predictions are okay, we can do a lot better
## Let's try few techniques that will unlock little more potential of ANNs

#### MLP ANN

In [ ]:
from tensorflow import keras
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization,Dropout
import datetime
start_time = datetime.datetime.now()

## Defining my optimizer - Its just a refined form of SGD
## ref - https://keras.io/optimizers/
adam = keras.optimizers.Adam(lr=0.01, decay=0.0005)

## Sequential model lets you add neural net layers one after another by calling function
model_mlp = keras.models.Sequential()

## Adding one layer having only 30 neuron
##mAdding one layer having only 30 neuron
## Notice our data has 8 input columns which goes into as the "input_shape" parameter
## Notice the use of l2 regularizer
model_mlp.add(keras.layers.Dense(30, input_shape=(8,), activation='relu'))
model_mlp.add(keras.layers.Dense(30, activation='relu'))
model_mlp.add(keras.layers.Dense(1, kernel_regularizer=keras.regularizers.l2()))


In [ ]:
## Callbacks
earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(patience=5, verbose=1)

## Compiling the model and defining the loss function
model_mlp.compile(optimizer=adam, loss='mse', metrics=['mae'])

## Training neural nets
model_mlp_history = model_mlp.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                          epochs=200, batch_size=15, callbacks=[reduce_lr, earlystopper])

end_time = datetime.datetime.now()
print(f'time taken for execution: {end_time-start_time} secs')

In [ ]:
model_mlp.summary()

In [ ]:
keras.utils.plot_model(model_mlp,rankdir='LR',show_shapes=True)

In [ ]:
# See the model structure
import pydot
tf.keras.utils.plot_model(model_mlp)

In [ ]:
from ann_visualizer.visualize import ann_viz
ann_viz(model_mlp,title="mlp_neural_network")

In [ ]:
mae_test = model_mlp.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(model_mlp_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

plt.show()

In [ ]:
X_new = X_test[:3]
y_pred = model_mlp.predict(X_new)
print(y_pred)
print(y_test[:3])

#### let's try the same with different params

In [ ]:
from tensorflow import keras
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization,Dropout

import datetime

start_time = datetime.datetime.now()

## Defining my optimizer - Its just a refined form of SGD
## ref - https://keras.io/optimizers/
adam = keras.optimizers.Adam(lr=0.001, decay=0.0005)

## Just a way to define neural nets. There are two ways sequential and functional
## Sequential model lets you add neural net layers one after another by calling function
model_norm = keras.models.Sequential()

## Adding one layer having only one neuron
## Notice our data has 8 input columns which goes into as the "input_shape" parameter
## Notice the use of l2 regularizer
model_norm.add(Dense(100, input_shape=(8,), activation='relu'))
model_norm.add(Dense(100, activation='relu'))
model_norm.add(Dense(1))


## Callbacks
earlystopper = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(patience=5, verbose=1)

## Compiling the model and defining loss function
model_norm.compile(optimizer=adam, loss='mse', metrics=['mae'])

## Training neural nets
model_norm_history = model_norm.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                          epochs=200, batch_size=50, callbacks=[reduce_lr, earlystopper])


end_time = datetime.datetime.now()
print(f'time taken for execution: {end_time-start_time} secs')

In [ ]:
# See the model structure
import pydot
keras.utils.plot_model(model_norm)

In [ ]:
keras.utils.plot_model(model_norm,rankdir='LR',show_shapes=True)

In [ ]:
mae_test = model_norm.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(model_norm_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)

plt.show()

In [ ]:
X_new = X_test[:3]
y_pred = model_norm.predict(X_new)
print(y_pred)
print(y_test[:3])